In [ ]:
!pip install datasets transformers torch
!pip install scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForSequenceClassification, DataCollatorWithPadding
from scipy.special import softmax
import os
from google.colab import drive
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

drive.mount('/content/drive')

os.environ['CUDA_VISIBLE_DEVICES'] = "0"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Modified preprocess function
def preprocess_function(examples):
    # Ensure 'text' is a list of strings
    texts = [str(text) for text in examples["text"]]
    return tokenizer(texts, truncation=True, padding=True)

# Load the single dataset
data_df = pd.read_csv("/content/drive/MyDrive/NLP Mental Health Detector/Datasets/data_to_be_cleansed.csv")

# Split the data into train, validation, and test sets
train_df, temp_df = train_test_split(data_df, test_size=0.4, random_state=42, stratify=data_df["target"])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df["target"])

# Create datasets from the dataframes
train_ds = Dataset.from_pandas(train_df[["text", "target"]])
val_ds = Dataset.from_pandas(val_df[["text", "target"]])
test_ds = Dataset.from_pandas(test_df[["text", "target"]])

# Tokenize the datasets
tokenized_train = train_ds.map(preprocess_function, batched=True)
tokenized_val = val_ds.map(preprocess_function, batched=True)
tokenized_test = test_ds.map(preprocess_function, batched=True)

# Drop the text column after tokenization
tokenized_train = tokenized_train.remove_columns(['text'])
tokenized_val = tokenized_val.remove_columns(['text'])
tokenized_test = tokenized_test.remove_columns(['text'])

# Initialize the data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Function to compute metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Variables to store metrics and predictions
acc = []
f1 = []
idx = []
preds = pd.DataFrame()

# Train the model for different experiments
for i in range(3, 4):
    idx.append(i)
    model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
    training_args = TrainingArguments(
        output_dir=f'./results/bert/bert-task1-exp-{i}',
        per_device_train_batch_size=32,
        num_train_epochs=15,
        learning_rate=0.00005,
        evaluation_strategy="epoch",  # Evaluate at the end of each epoch
        save_strategy="epoch",        # Save the model at the end of each epoch
        load_best_model_at_end=True,  # Load the best model at the end of training
        metric_for_best_model="f1"    # Use F1 score to determine the best model
    )

    # Trainer object initialization
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

    # Evaluate on validation set
    eval_results = trainer.evaluate()
    print(f"Validation Results: {eval_results}")

    # Evaluate on test set
    test_results = trainer.predict(tokenized_test)
    print(f"Test Results: {test_results.metrics}")

    # Detailed classification report
    test_preds = np.argmax(test_results.predictions, axis=1)
    test_labels = test_results.label_ids
    print(classification_report(test_labels, test_preds))

    # Save the model
    model.save_pretrained(f"./models_bayes/bert-task1-exp-{i}")

    # Make predictions on validation set
    pred = trainer.predict(tokenized_val)
    pred_train = trainer.predict(tokenized_train)

    preds_test = pd.DataFrame(np.concatenate([softmax(pred.predictions, axis=1), pred.label_ids.reshape((-1, 1))], axis=1), columns=['0', '1', '2', 'label'])
    preds_train = pd.DataFrame(np.concatenate([softmax(pred_train.predictions, axis=1), pred_train.label_ids.reshape((-1, 1))], axis=1), columns=['0', '1', '2', 'label'])
    preds_test.to_csv(f"./predictions_bayes/bert-task1-exp-{i}-preds-val.csv", index=False)
    preds_train.to_csv(f"./predictions_bayes/bert-task1-exp-{i}-preds-train.csv", index=False)

    model_predictions = np.argmax(softmax(pred.predictions, axis=1), axis=1)
    preds["target"] = model_predictions
    preds.to_csv(f"./predictions_bayes/bert-task1-exp-{i}-preds.csv", index=False)

    # Store metrics
    acc.append(eval_results['eval_accuracy'])
    f1.append(eval_results['eval_f1'])

# Print overall results
print("\nOverall Results:")
for i, (accuracy, f1_score) in enumerate(zip(acc, f1), start=3):
    print(f"Experiment {i}: Accuracy = {accuracy:.4f}, F1 Score = {f1_score:.4f}")